In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Random Forests

In [5]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [6]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [7]:
randomforest = RandomForestRegressor()
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 86.83 % , error = 18173048430.320446


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [8]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [9]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [22]:
lista_arboles = [18,19,21,22,23,24,25]
lista_feature = [0.4,0.5]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((cant_arbol,feature,precision,error))
        print(cant_arbol,' - ',feature)

18  -  0.4
18  -  0.5
19  -  0.4
19  -  0.5
21  -  0.4
21  -  0.5
22  -  0.4
22  -  0.5
23  -  0.4
23  -  0.5
24  -  0.4
24  -  0.5
25  -  0.4
25  -  0.5


In [23]:
for r in res:
    print("Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Cant arboles = 5, % features = 0.7, precision = 84.56 % , error = 21303283909.34756
Cant arboles = 5, % features = 0.8, precision = 86.98 % , error = 17965657840.70977
Cant arboles = 5, % features = 1.0, precision = 85.78 % , error = 19620180384.3387
Cant arboles = 10, % features = 0.7, precision = 87.80 % , error = 16829288107.881966
Cant arboles = 10, % features = 0.8, precision = 88.25 % , error = 16211514021.839184
Cant arboles = 10, % features = 1.0, precision = 87.08 % , error = 17828865003.33896
Cant arboles = 15, % features = 0.7, precision = 88.11 % , error = 16405965237.87476
Cant arboles = 15, % features = 0.8, precision = 87.22 % , error = 17635545323.111412
Cant arboles = 15, % features = 1.0, precision = 87.26 % , error = 17575375715.987404
Cant arboles = 20, % features = 0.7, precision = 87.63 % , error = 17063372312.071825
Cant arboles = 20, % features = 0.8, precision = 87.91 % , error = 16677076812.98315
Cant arboles = 20, % features = 1.0, precision = 86.92 % , error

In [24]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Cant arboles = 20, % features = 0.4, precision = 88.63 % , error = 15692906830.660103
Menor error = Cant arboles = 20, % features = 0.4, precision = 88.63 % , error = 15692906830.660103


## Tomamos features = 0.4 y  cant = 20

In [39]:
#Probamos con varias combinaciones de columnas, y esta fue la de mejores resultados

columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [40]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [41]:
cant_arbol = 20
feature = 0.4

In [42]:
randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 88.21 % , error = 16268143364.99585


# calculo con los verdaderos datos a analizar

In [43]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [44]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [45]:
resultado = analizar.loc[:,['id','price_usd']]

In [46]:
resultado.to_csv('Random_Forests_resultados', index = False)

## Utilizando standard scaler

In [6]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [7]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [8]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(set_entrenamiento_datos)  

StandardScaler(copy=True, with_mean=True, with_std=True)

In [37]:
randomforest = RandomForestRegressor(n_estimators = 20, max_features = 0.4, warm_start = True)
for i in range (15):
    randomforest.fit(scaler.transform(set_entrenamiento_datos),set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(scaler.transform([x]))[0],axis = 1)
precision = randomforest.score(scaler.transform(set_pruebas.loc[:,columnas]),set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 88.38 % , error = 16025887536.8


In [38]:
analizar = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/properati_dataset_modificado.csv')

In [39]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(scaler.transform([x]))[0],axis = 1)

In [41]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.427062e+05
std      3.100394e+05
min      5.005030e+03
25%      1.065692e+05
50%      1.597793e+05
75%      2.700639e+05
max      2.037744e+07
Name: price_usd, dtype: float64

In [42]:
resultado = analizar.loc[:,['id','price_usd']]

In [43]:
resultado.to_csv('resultados/Random_Forests_standard_scaler_resultados', index = False)

# Usando lat lon en lugar de place name

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [6]:
randomforest = RandomForestRegressor()
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 95.02 % , error = 5769331856.080151


In [7]:
res = []

In [8]:
lista_arboles = [5,10,15,20,25,30,50]
lista_feature = [0.4,0.5,0.6,0.8,1.0]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((cant_arbol,feature,precision,error))
        print(cant_arbol,' - ',feature)

5  -  0.4
5  -  0.5
5  -  0.6
5  -  0.8
5  -  1.0
10  -  0.4
10  -  0.5
10  -  0.6
10  -  0.8
10  -  1.0
15  -  0.4
15  -  0.5
15  -  0.6
15  -  0.8
15  -  1.0
20  -  0.4
20  -  0.5
20  -  0.6
20  -  0.8
20  -  1.0
25  -  0.4
25  -  0.5
25  -  0.6
25  -  0.8
25  -  1.0
30  -  0.4
30  -  0.5
30  -  0.6
30  -  0.8
30  -  1.0
50  -  0.4
50  -  0.5
50  -  0.6
50  -  0.8
50  -  1.0


In [9]:
for r in res:
    print("Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Cant arboles = 5, % features = 0.4, precision = 94.55 % , error = 6307030644.998087
Cant arboles = 5, % features = 0.5, precision = 93.82 % , error = 7154235562.696608
Cant arboles = 5, % features = 0.6, precision = 95.77 % , error = 4902160429.393464
Cant arboles = 5, % features = 0.8, precision = 93.82 % , error = 7158688782.447428
Cant arboles = 5, % features = 1.0, precision = 95.51 % , error = 5199701473.432254
Cant arboles = 10, % features = 0.4, precision = 95.66 % , error = 5024785549.323212
Cant arboles = 10, % features = 0.5, precision = 94.52 % , error = 6339578504.01168
Cant arboles = 10, % features = 0.6, precision = 94.95 % , error = 5844629858.660479
Cant arboles = 10, % features = 0.8, precision = 95.02 % , error = 5770632726.046741
Cant arboles = 10, % features = 1.0, precision = 95.13 % , error = 5634883047.226112
Cant arboles = 15, % features = 0.4, precision = 96.69 % , error = 3834602573.188404
Cant arboles = 15, % features = 0.5, precision = 96.24 % , error = 4350

In [10]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if r[2] > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Cant arboles = 15, % features = 0.4, precision = 96.69 % , error = 3834602573.188404
Menor error = Cant arboles = 15, % features = 0.4, precision = 96.69 % , error = 3834602573.188404


## Tomamos features = 0.4 y  cant = 15

In [30]:
#Probamos distintas combinaciones de columnas, esta es la que dio mejores resultados

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [31]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [32]:
cant_arbol = 15
feature = 0.4

In [33]:
randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 95.78 % , error = 4890492521.827894


# calculo con los verdaderos datos a analizar

In [34]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [35]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [36]:
resultado = analizar.loc[:,['id','price_usd']]

In [37]:
resultado.to_csv('resultados/Random_Forests_2', index = False)

# Usando Grid Search y Cross Validation

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [3]:
# Parametros

lista_arboles = [5,10,15,20,30,50]
lista_feature = [0.4,0.5,0.7,1.0]

parametros = {"n_estimators" : lista_arboles, "max_features" : lista_feature}

iteraciones_cross_validation = 10
rf = RandomForestRegressor()

In [4]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = rf, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:25:17 --- 20:16:41 
 Precision: 93.72 
 Parametros = {'max_features': 0.4, 'n_estimators': 20}


In [5]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict(x)[0],axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Random_Forests_GridSearch.csv', index = False)

joblib.dump(grid, 'algoritmos/random_forests.pkl')

['algoritmos/random_forests.pkl']

## Hago Bagging con el mejor algoritmo

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [3]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    rf = RandomForestRegressor(n_estimators = 20, max_features = 0.4)
    
    rf.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: rf.predict(x)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [4]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Random_Forests_Cross_Validation.csv', index = False)